<a href="https://colab.research.google.com/github/bjungweapon/mjc.ai.ml/blob/BDU/BDU_L2_L1_Retriction_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# 1. 데이터 로드 (diabetes 데이터셋 또는 사용자 정의 CSV 파일)
# -------------------------------------------------------------------
# 사이킷런 내장 diabetes 데이터셋 사용 (간단한 예시)
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
target = pd.Series(diabetes.target)

In [ ]:
data

In [ ]:
data.info()

방금 코드에서 사용한 diabetes 데이터셋은 당뇨병 연구를 위해 수집된 의료 데이터를 담고 있습니다. 이 데이터셋은 주로 다음과 같은 정보를 포함하고 있습니다.

진단 정보: 환자의 당뇨병 발병 여부 (종속 변수).
생체 측정 데이터:
혈압
혈당 수치
BMI (Body Mass Index, 체질량 지수)
피부 두께
인슐린 수치 등
기타 의료 정보:
임신 횟수
나이
이 데이터셋은 주로 회귀 분석 모델을 학습하여 환자의 특정 생체 측정 및 의료 정보를 기반으로 당뇨병과 관련된 특정 지표를 예측하는 데 사용됩니다. 예를 들어, 혈당 수치, BMI 등이 주어졌을 때 당뇨병의 진행 정도를 예측하는 등의 분석에 활용될 수 있습니다.

sklearn.datasets.load_diabetes()로 불러온 데이터는 이미 어느 정도의 전처리 과정을 거친 데이터입니다. 특히, 각 특성(feature)은 **평균이 0, 표준편차가 1에 가깝도록 표준화(Standardization)**되어 있습니다.

따라서 data DataFrame의 age 컬럼에서 음수 값이 나타나는 것은 원래 나이 값에서 평균 나이를 빼고 표준편차로 나누는 표준화 과정을 거쳤기 때문입니다. 표준화된 값은 원래 값의 크기와 단위를 잃게 되며, 평균보다 작은 값은 음수로, 큰 값은 양수로 표현됩니다.



2. data 분할

In [ ]:
# 2. 데이터 분할 (학습 데이터와 테스트 데이터)
# -------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)


3. data scaling

In [ ]:
# 3. 데이터 스케일링 (L1, L2 규제는 스케일에 민감하므로 중요)
# -------------------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

4. L1 규제 적용

In [ ]:
# 4. L1 규제 (Lasso) 모델 학습 및 평가
# -------------------------------------------------------------------
alpha_lasso = 0.1  # 규제 강도 (하이퍼파라미터)
lasso = Lasso(alpha=alpha_lasso)
lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_test_scaled)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
print(f"Lasso (L1 규제) Mean Squared Error: {mse_lasso:.4f}")
print("\nLasso (L1 규제) Coefficient (절댓값이 0에 가까운 특징은 영향력이 작아짐):")
for name, coef in zip(data.columns, lasso.coef_):
    print(f"{name}: {coef:.4f}")

영향력이 큰 특성:

bmi (체질량 지수): 25.8246 - 계수의 절댓값이 비교적 큰 양수이므로, 체질량 지수가 높을수록 당뇨병 지표 값이 증가하는 경향이 있으며, 예측에 미치는 영향이 크다고 해석할 수 있습니다.
s5 (혈청 트리글리세리드 수치 log scale): 29.6328 - 계수의 절댓값이 가장 큰 양수이므로, 혈청 트리글리세리드 수치가 높을수록 당뇨병 지표 값이 크게 증가하며, 예측에 가장 큰 영향을 미치는 특성이라고 볼 수 있습니다.
s1 (총 콜레스테롤 수치): -29.3584 - 계수의 절댓값이 큰 음수이므로, 총 콜레스테롤 수치가 높을수록 당뇨병 지표 값이 크게 감소하는 경향이 있으며, 예측에 상당한 영향을 미치는 특성입니다.
영향력이 중간 정도인 특성:

bp (평균 혈압): 16.6443 - 계수의 절댓값이 중간 정도의 양수이므로, 평균 혈압이 높을수록 당뇨병 지표 값이 증가하는 경향이 있으며, 예측에 어느 정도 영향을 미칩니다.
s2 (LDL 콜레스테롤): 13.2758 - 계수의 절댓값이 중간 정도의 양수이므로, LDL 콜레스테롤 수치가 높을수록 당뇨병 지표 값이 증가하는 경향이 있으며, 예측에 어느 정도 영향을 미칩니다.
s4 (총 콜레스테롤/HDL 비율): 10.2362 - 계수의 절댓값이 중간 정도의 양수이므로, 총 콜레스테롤/HDL 비율이 높을수록 당뇨병 지표 값이 증가하는 경향이 있으며, 예측에 어느 정도 영향을 미칩니다.
age (나이): 1.7305 - 계수의 절댓값이 비교적 작은 양수이므로, 나이가 많을수록 당뇨병 지표 값이 약간 증가하는 경향이 있을 수 있지만, 다른 특성에 비해 영향력은 작다고 볼 수 있습니다.
s6 (혈당 수치): 2.3935 - 계수의 절댓값이 비교적 작은 양수이므로, 혈당 수치가 높을수록 당뇨병 지표 값이 약간 증가하는 경향이 있을 수 있지만, 다른 특성에 비해 영향력은 작다고 볼 수 있습니다.
s3 (HDL 콜레스테롤): 0.5479 - 계수의 절댓값이 매우 작은 양수이므로, HDL 콜레스테롤 수치가 높을수록 당뇨병 지표 값이 아주 약간 증가하는 경향이 있을 수 있지만, 예측에 미치는 영향은 매우 작다고 판단할 수 있습니다.
음의 영향:

sex (성별): -11.3164 - 계수가 음수이므로, 특정 성별(데이터 인코딩 방식에 따라 남성 또는 여성)일수록 당뇨병 지표 값이 감소하는 경향이 있음을 나타냅니다.
종합적인 해석:

Lasso 규제 결과, **체질량 지수(bmi), 혈청 트리글리세리드 수치(s5), 총 콜레스테롤 수치(s1)**가 당뇨병 지표 값에 가장 큰 영향을 미치는 주요 요인으로 보입니다. 평균 혈압(bp), LDL 콜레스테롤(s2), 총 콜레스테롤/HDL 비율(s4)도 어느 정도 영향을 미치는 것으로 나타났습니다.

반면에 나이(age), 혈당 수치(s6), HDL 콜레스테롤(s3)은 상대적으로 영향력이 작거나 미미한 것으로 보입니다. 성별(sex)은 당뇨병 지표 값에 음의 상관관계를 가질 수 있음을 시사합니다.

주의할 점:

데이터 인코딩: 범주형 변수인 'sex'의 경우, 어떻게 수치로 인코딩되었는지(예: 0과 1이 각각 어떤 성별을 나타내는지) 알아야 정확한 해석이 가능합니다.
상관관계: 특성들 간의 상관관계를 고려해야 합니다. 예를 들어, 콜레스테롤 관련 특성들(s1, s2, s3, s4)은 서로 높은 상관관계를 가질 수 있으며, L1 규제는 그중 일부를 선택하고 나머지의 계수를 0으로 만들 수 있습니다.
규제 강도(alpha): 사용된 alpha 값에 따라 계수 값과 선택되는 특성이 달라질 수 있습니다. 다른 alpha 값으로 실험해보면 특성 중요도에 대한 더 robust한 결론을 얻을 수 있습니다.
인과관계: 계수 값은 특성과 종속 변수 간의 상관관계를 나타낼 뿐, 인과관계를 의미하는 것은 아닙니다.
이러한 점들을 고려하여 결과를 해석하고, 실제 도메인 지식과 결합하여 더 깊이 있는 인사이트를 도출할 수 있습니다.

5. L2 규제

In [ ]:
# 5. L2 규제 (Ridge) 모델 학습 및 평가
# -------------------------------------------------------------------
alpha_ridge = 1.0  # 규제 강도 (하이퍼파라미터)
ridge = Ridge(alpha=alpha_ridge)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print(f"\nRidge (L2 규제) Mean Squared Error: {mse_ridge:.4f}")
print("\nRidge (L2 규제) Coefficient (계수 크기를 전반적으로 줄임):")
for name, coef in zip(data.columns, ridge.coef_):
    print(f"{name}: {coef:.4f}")

제공해주신 L2 규제(Ridge) 결과를 바탕으로 인사이트를 해석해 보겠습니다. 또한 이전의 L1 규제(Lasso) 결과와 비교하여 어떤 차이점과 유사점을 보이는지 설명드리겠습니다.

L2 규제 (Ridge) 결과 해석:

age (나이): 1.8073 - 양의 계수로, 나이가 증가할수록 당뇨병 지표 값이 소폭 증가하는 경향을 보입니다. L1 결과와 유사하게 영향력은 크지 않습니다.
sex (성별): -11.4482 - 음의 계수로, 특정 성별(인코딩 방식에 따라 다름)일수록 당뇨병 지표 값이 감소하는 경향을 보입니다. L1 결과와 비슷한 크기의 영향력을 나타냅니다.
bmi (체질량 지수): 25.7327 - 양의 계수로, 체질량 지수가 높을수록 당뇨병 지표 값이 증가하는 경향을 보입니다. L1 결과와 비슷한 크기로, 여전히 중요한 예측 변수임을 시사합니다.
bp (평균 혈압): 16.7343 - 양의 계수로, 평균 혈압이 높을수록 당뇨병 지표 값이 증가하는 경향을 보입니다. L1 결과와 유사한 영향력을 가집니다.
s1 (총 콜레스테롤 수치): -34.6720 - 음의 계수로, 총 콜레스테롤 수치가 높을수록 당뇨병 지표 값이 감소하는 경향을 보입니다. L1 결과보다 절댓값이 약간 더 커진 것으로 보아, L2 규제 하에서도 유의미한 영향력을 갖는 변수입니다.
s2 (LDL 콜레스테롤): 17.0531 - 양의 계수로, LDL 콜레스테롤 수치가 높을수록 당뇨병 지표 값이 증가하는 경향을 보입니다. L1 결과보다 계수 크기가 커진 것을 확인할 수 있습니다.
s3 (HDL 콜레스테롤): 3.3699 - 양의 계수로, HDL 콜레스테롤 수치가 높을수록 당뇨병 지표 값이 소폭 증가하는 경향을 보입니다. L1 결과에서는 매우 작은 양수였던 것에 비해 약간 증가했지만, 여전히 다른 주요 변수들에 비해 영향력은 작습니다.
s4 (총 콜레스테롤/HDL 비율): 11.7643 - 양의 계수로, 총 콜레스테롤/HDL 비율이 높을수록 당뇨병 지표 값이 증가하는 경향을 보입니다. L1 결과와 비슷한 수준의 영향력을 보입니다.
s5 (혈청 트리글리세리드 수치 log scale): 31.3784 - 양의 계수로, 혈청 트리글리세리드 수치가 높을수록 당뇨병 지표 값이 증가하는 경향을 보입니다. L1 결과와 마찬가지로 가장 큰 양의 영향력을 갖는 변수 중 하나입니다.
s6 (혈당 수치): 2.4581 - 양의 계수로, 혈당 수치가 높을수록 당뇨병 지표 값이 소폭 증가하는 경향을 보입니다. L1 결과와 비슷한 정도로 영향력은 작습니다.
L1 규제 결과와의 비교를 통한 인사이트:

계수가 0인 특성 부재: L1 규제와 달리, L2 규제는 계수를 정확히 0으로 만들지 않습니다. 모든 특성이 모델 예측에 기여하지만, 그 영향력의 크기는 다릅니다. 이는 L2 규제가 특성 선택보다는 계수 크기를 전반적으로 줄이는 데 초점을 맞추기 때문입니다.
전반적으로 유사한 방향성: 대부분의 특성에서 L1과 L2 규제 모두 계수의 부호(양수 또는 음수)가 동일합니다. 이는 각 특성이 당뇨병 지표에 미치는 기본적인 경향이 유사하다는 것을 의미합니다. 예를 들어, bmi와 s5는 양의 영향을, s1은 음의 영향을 미치는 경향이 दोनों 모델에서 나타납니다.
계수 크기의 차이: L1 규제는 일부 계수를 0으로 만들면서 나머지 계수의 크기를 상대적으로 덜 줄이는 경향이 있는 반면, L2 규제는 모든 계수의 크기를 전반적으로 줄입니다. 따라서 L1 결과에서 0이 아니었던 계수들의 절댓값이 L2 결과에서는 다소 작아지거나 커진 것을 확인할 수 있습니다. 특히, L1에서 상대적으로 영향력이 작았던 특성(예: s3)의 계수가 L2에서는 0이 아닌 작은 값으로 유지되는 것을 볼 수 있습니다.
모델 복잡성: L1 규제는 일부 계수를 0으로 만들어 모델을 더 단순하게 만드는 효과가 있지만, L2 규제는 모든 특성을 유지하므로 모델이 더 복잡할 수 있습니다.
MSE 비교: 제공해주신 L2 규제의 평균 제곱 오차(MSE)는 2892.0146입니다. L1 규제의 MSE는 코드 실행 결과에 따라 다르겠지만, 이 값을 비교하여 어떤 규제가 현재 데이터에 더 적합한 성능을 보이는지 판단할 수 있습니다. 일반적으로 L1은 특성 선택에, L2는 과적합 완화에 더 효과적이라고 알려져 있지만, 실제 데이터에 따라 결과는 달라질 수 있습니다.
결론적으로, L2 규제 결과는 모든 특성이 당뇨병 지표 예측에 기여하지만, 체질량 지수(bmi), 혈청 트리글리세리드 수치(s5), 총 콜레스테롤 수치(s1)가 여전히 중요한 예측 변수임을 보여줍니다. L1 규제와 비교했을 때, 특성 선택의 효과는 없지만, 계수 크기를 전반적으로 줄여 모델의 과적합을 완화하는 데 기여할 수 있습니다. 어떤 규제가 더 나은지는 모델의 성능 평가 지표(예: MSE)를 비교하여 결정하는 것이 중요합니다.

{ 분석 공식 ]


L1 또는 L2 규제를 적용한 회귀 분석 결과에서 나타나는 각 특성의 계수 값은 다음과 같은 의미로 해석할 수 있습니다.

큰 양수 값: 해당 특성의 값이 증가할수록 종속 변수(예: 당뇨병 지표)의 값도 크게 증가하는 경향이 있음을 나타냅니다. 이는 강한 양의 상관관계를 의미합니다.
큰 음수 값: 해당 특성의 값이 증가할수록 종속 변수의 값은 크게 감소하는 경향이 있음을 나타냅니다. 이는 강한 음의 상관관계를 의미합니다.
0에 가까운 값: 해당 특성의 변화가 종속 변수의 변화에 미치는 영향이 매우 작거나 거의 없다는 것을 의미합니다. 따라서 종속 변수와의 상관관계가 약하다고 해석할 수 있습니다. 특히 L1 규제에서는 계수가 정확히 0이 되는 경우 해당 특성이 모델 예측에 거의 영향을 미치지 않는다고 볼 수 있습니다.
정리하자면:

계수의 부호는 특성과 종속 변수 간의 상관관계의 방향 (양의 상관 또는 음의 상관)을 나타냅니다.
계수의 절댓값 크기는 특성과 종속 변수 간의 상관관계의 강도를 나타냅니다. 절댓값이 클수록 상관관계가 강하고, 0에 가까울수록 상관관계가 약합니다.
이러한 해석을 통해 어떤 특성이 종속 변수에 얼마나 그리고 어떤 방향으로 영향을 미치는지 파악하고, 데이터에 대한 중요한 인사이트를 얻을 수 있습니다.



6. 결과 시각화

In [ ]:
# 6. 결과 시각화 (예시: 실제 값 vs 예측 값)
# -------------------------------------------------------------------
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_lasso)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Value')
plt.ylabel('Predicted Value (Lasso)')
plt.title(f'Lasso Regression (alpha={alpha_lasso:.2f})')

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_ridge)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Value')
plt.ylabel('Predicted Value (Ridge)')
plt.title(f'Ridge Regression (alpha={alpha_ridge:.2f})')

plt.tight_layout()
plt.show()

7.  알파 (계수) 변화에 따른 시각화

In [ ]:
# 7. 규제 강도(alpha)에 따른 계수 변화 시각화 (선택 사항)
# -------------------------------------------------------------------
alphas = [0.001, 0.01, 0.1, 1.0, 10.0]
lasso_coefs = []
ridge_coefs = []

for alpha in alphas:
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train_scaled, y_train)
    lasso_coefs.append(lasso_model.coef_)

    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train_scaled, y_train)
    ridge_coefs.append(ridge_model.coef_)

plt.figure(figsize=(12, 8))

plt.subplot(1, 2, 1)
for coef, feature in zip(zip(*lasso_coefs), data.columns):
    plt.plot(alphas, coef, label=feature)
plt.xscale('log')
plt.xlabel('Alpha (Lasso)')
plt.ylabel('Coefficients')
plt.title('Lasso Coefficients vs Alpha')
plt.legend(loc='best')
plt.grid(True)

plt.subplot(1, 2, 2)
for coef, feature in zip(zip(*ridge_coefs), data.columns):
    plt.plot(alphas, coef, label=feature)
plt.xscale('log')
plt.xlabel('Alpha (Ridge)')
plt.ylabel('Coefficients')
plt.title('Ridge Coefficients vs Alpha')
plt.legend(loc='best')
plt.grid(True)

plt.tight_layout()
plt.show()

하지만 s1 특성에서 alpha 값이 증가함에도 불구하고 계수의 절댓값이 증가하는 현상은 **다중공선성(Multicollinearity)**의 가능성을 강하게 시사합니다.

다중공선성이란?

다중공선성은 예측 변수(여기서는 특성)들 간에 높은 상관관계가 존재할 때 발생합니다. 이러한 상황에서는 모델이 특정 예측 변수의 영향을 정확하게 추정하기 어려워지고, 계수 값이 불안정해지는 경향이 있습니다.

s1 특성에서 나타나는 현상에 대한 해석:

다른 특성과의 강한 상관관계: s1 특성이 다른 하나 이상의 특성과 매우 높은 상관관계를 가지고 있을 가능성이 큽니다.

규제 변화에 따른 영향력 변화:

alpha 값이 작을 때 (규제가 약할 때), 모델은 상관관계가 높은 여러 특성들에 분산하여 가중치를 할당할 수 있습니다. 이때 s1의 계수는 상대적으로 작을 수 있습니다.
alpha 값이 증가하면 (규제가 강해지면), 모델은 불필요한 변수의 가중치를 0 또는 0에 가깝게 만들려고 시도합니다. 이 과정에서 s1과 강한 상관관계를 갖는 다른 특성들의 가중치가 먼저 감소하거나 0이 될 수 있습니다.
결과적으로, 모델이 종속 변수를 설명하는 데 s1의 중요성을 더 크게 인식하게 되고, 규제가 강해짐에도 불구하고 오히려 s1에 더 큰 가중치를 부여하게 될 수 있습니다. 이는 모델이 "어쩔 수 없이" s1을 더 많이 활용하여 예측을 수행하려는 경향으로 나타날 수 있습니다.
L1, L2 모두에서 나타나는 이유: 다중공선성의 문제는 규제의 종류와 상관없이 모델의 계수 추정에 영향을 미칠 수 있습니다. L1 규제는 일부 계수를 0으로 만들면서 이러한 현상을 다소 완화할 수 있지만, 여전히 남아있는 특성들의 계수 변화는 불안정할 수 있습니다. L2 규제는 모든 계수를 0에 가깝게 만들려고 하지만, 상관관계가 높은 특성들 사이에서 가중치가 조절되는 방식이 복잡하게 나타날 수 있습니다.

확인 및 해결 방법:

상관관계 분석: pandas의 .corr() 메서드를 사용하여 특성들 간의 상관관계 행렬을 확인해 보세요. s1과 높은 상관관계를 갖는 다른 특성이 있는지 확인하는 것이 중요합니다.
VIF (Variance Inflation Factor) 확인: 다중공선성의 정도를 정량적으로 측정하는 VIF를 계산해 볼 수 있습니다. 일반적으로 VIF 값이 10 이상이면 다중공선성이 심각하다고 판단합니다.
특성 제거 또는 병합: 다중공선성이 심각한 경우, 상관관계가 높은 특성 중 하나를 제거하거나, 여러 특성을 조합하여 새로운 특성을 만드는 것을 고려해 볼 수 있습니다.
PCA (Principal Component Analysis) 등의 차원 축소: 다중공선성을 해결하고 데이터의 차원을 축소하는 방법도 고려해 볼 수 있습니다.
결론적으로, alpha 값이 증가함에도 불구하고 s1의 계수 절댓값이 증가하는 현상은 s1이 다른 특성들과 강한 다중공선성 관계를 가지고 있을 가능성이 매우 높다는 것을 시사합니다.